### Задача 2  

Вам доступны следующие фреймы данных: 

- Таблица с информацией о среднедневном спросе на каждый товар для каждой локации:  

| product | location | demand |
|---|---|---|
| 1 | 01 | 100 |
| 1 | 02 | 110 |
| 2 | 01 | 120 |
| 2 | 02 | 90 |
| 3 | 01 | 70 |
| 3 | 02 | 80 |

`product` — уникальный идентификатор продукта  
`location` — уникальный идентификатор локации  
`demand` — значение среднедневного спроса на конкретный товар в конкретной локации в единицах товара  

- Таблица с информацией о складских запасах на уровне месяца:  

| product | location | stock |
|---|---|---|
| 1 | 01 | 1000 |
| 1 | 02 | 400 |
| 2 | 01 | 300 |
| 2 | 02 | 250 |

`product` — уникальный идентификатор продукта  
`stock` — уровень запасов в единицах товара на уровне месяца  

В качестве примера рассмотрим июнь 2023 года. Вам необходимо сформировать витрину данных, в которой для каждой пары товар-локация на уровне каждой технической недели* будет рассчитано прогнозируемое значение количества проданных товаров (с учетом среднедневного спроса) и количество остатков товара на складе. 

Техническая неделя — это календарная неделя или часть календарной недели, которая «укладывается» в один календарный месяц. Например, в августе 2023 года вам доступны следующие технические недели: 

01.08—06.08

07.08—13.08

14.08—20.08

21.08—27.08

28.08—31.08


In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\User\\anaconda3\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[8]")\
    .appName("SparkFirst")\
    .config("spark.executor.memory", "10g")\
    .config("spark.executor.cores", 5)\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.maxExecutors", 5)\
    .config("spark.shuffle.service.enabled", "true")\
    .getOrCreate()

In [12]:
import pandas as pd

avg_day_demand_schema = StructType([
    StructField('product', IntegerType(), True),
    StructField('location', StringType(), True),
    StructField('demand', IntegerType(), True)
])

product_stock_schema = StructType([
    StructField('product', IntegerType(), True),
    StructField('location', StringType(), True),
    StructField('stock', IntegerType(), True)
])

avg_day_demand_data = [
    (1, "01", 100),
    (1, "02", 110),
    (2, "01", 120),
    (2, "02", 90),
    (3, "01", 70),
    (3, "02", 80)
]

product_stock_data = [
    (1, "01", 1000),
    (1, "02", 400),
    (2, "01", 300),
    (2, "02", 250)
]


avg_day_demand_df = spark.createDataFrame(
    data=avg_day_demand_data, schema=avg_day_demand_schema)
product_stock_df = spark.createDataFrame(
    data=product_stock_data, schema=product_stock_schema)

In [13]:
avg_day_demand_df.createOrReplaceTempView("avg_day_demand")
avg_day_demand_df.show()

+-------+--------+------+
|product|location|demand|
+-------+--------+------+
|      1|      01|   100|
|      1|      02|   110|
|      2|      01|   120|
|      2|      02|    90|
|      3|      01|    70|
|      3|      02|    80|
+-------+--------+------+



In [14]:
product_stock_df.createOrReplaceTempView("product_stock")
product_stock_df.show()

+-------+--------+-----+
|product|location|stock|
+-------+--------+-----+
|      1|      01| 1000|
|      1|      02|  400|
|      2|      01|  300|
|      2|      02|  250|
+-------+--------+-----+



In [15]:
from datetime import date, timedelta

d1 = date(2023, 8, 1)  # начальная дата
d2 = date(2023, 9, 1)  # конечная дата

delta = d2 - d1        # timedelta

calendar_data = pd. DataFrame([(d1 + timedelta(i)).strftime("%Y-%m-%d")
                              for i in range(delta.days)], columns=['date'])
calendar_df = spark.createDataFrame(calendar_data)
calendar_df.createOrReplaceTempView("calendar")
calendar_df.show()

+----------+
|      date|
+----------+
|2023-08-01|
|2023-08-02|
|2023-08-03|
|2023-08-04|
|2023-08-05|
|2023-08-06|
|2023-08-07|
|2023-08-08|
|2023-08-09|
|2023-08-10|
|2023-08-11|
|2023-08-12|
|2023-08-13|
|2023-08-14|
|2023-08-15|
|2023-08-16|
|2023-08-17|
|2023-08-18|
|2023-08-19|
|2023-08-20|
+----------+
only showing top 20 rows



In [16]:

technical_weeks_df = spark.sql("""
WITH calendar_weeks AS (
    select
        date,
        WEEKOFYEAR(to_date(cast(date as date), "yyyy-mm-dd")) as week_number
    from calendar
),
calendar_days_in_week AS (
    select
        week_number,
        count(week_number) OVER (partition by week_number) as day_in_week
    from calendar_weeks
)
select
    week_number,
    day_in_week
from calendar_days_in_week
group by week_number, day_in_week
""")
technical_weeks_df.createOrReplaceTempView("technical_weeks")
technical_weeks_df.show()

+-----------+-----------+
|week_number|day_in_week|
+-----------+-----------+
|         31|          6|
|         32|          7|
|         33|          7|
|         34|          7|
|         35|          4|
+-----------+-----------+



In [21]:
recency = spark.sql("""
WITH 1temp AS (
    SELECT
    avg_day_demand.product as product,
    avg_day_demand.location as location,
    avg_day_demand.demand as demand,
    product_stock.stock as stock
    FROM avg_day_demand
    JOIN product_stock ON product_stock.location = avg_day_demand.location AND product_stock.product = avg_day_demand.product
),
2temp AS (
    select 
        *,
        sum(demand * day_in_week) OVER (PARTITION BY product, location ORDER BY week_number) as demand_sum,
        stock - sum(demand * day_in_week) OVER (PARTITION BY product, location ORDER BY week_number) as stock_balance
    from technical_weeks
    join 1temp
)
select 
    week_number as W,
    product as P,
    location as L,
    demand * day_in_week as WD,
    demand_sum as CWD,
    stock as SPM,
    CASE
        WHEN stock_balance + (demand * day_in_week) > 0 THEN stock_balance + (demand * day_in_week) 
        ELSE 0
    END as SWB,
    CASE
        WHEN stock_balance > 0 THEN stock_balance 
        ELSE 0
    END as SWE,
    CASE
        WHEN stock_balance >= demand THEN 0
        WHEN stock_balance <= demand THEN stock_balance + (demand * day_in_week) - demand * day_in_week
        ELSE 0
    END as SS,
    CASE
        WHEN stock_balance > 0  AND demand <= stock_balance THEN demand * day_in_week
        WHEN stock_balance > 0  AND demand >= stock_balance THEN demand * day_in_week
        WHEN stock_balance + demand * day_in_week > 0 THEN stock_balance + demand * day_in_week
        ELSE 0
    END as sales_forecast
from 2temp
ORDER BY product, location, week_number 

""")
print('"W"   - "week" - номер недели \n'
     + '"P"   - "product" - id продукта \n'
     + '"L"   - "location" - id локации \n'
     + '"WD"  - "weekly_demand" - недельный спрос на товар в локации \n'
     + '"CWD" - "cumulative_weekly_demand" - недельный спрос на товар в локации, нарастающим итогом \n'
     + '"SPM" - "stock_per_month" - товарный запас на месяц \n'
     + '"SWB" - "stock_week_beginning" - товарный запас на начало недели \n'
     + '"SWE" - "stock_week_end" - товарный запас на конец недели \n'
     + '"SS"  - "stock_shortage" - дефицит товара на конец недели  \n'
     + '"sales_forecast" - прогнозируемое значение количества проданных товаров \n')
recency.show()

"W"   - "week" - номер недели 
"P"   - "product" - id продукта 
"L"   - "location" - id локации 
"WD"  - "weekly_demand" - недельный спрос на товар в локации 
"CWD" - "cumulative_weekly_demand" - недельный спрос на товар в локации, нарастающим итогом 
"SPM" - "stock_per_month" - товарный запас на месяц 
"SWB" - "stock_week_beginning" - товарный запас на начало недели 
"SWE" - "stock_week_end" - товарный запас на конец недели 
"SS"  - "stock_shortage" - дефицит товара на конец недели  
"sales_forecast" - прогнозируемое значение количества проданных товаров 

+---+---+---+---+----+----+----+---+-----+--------------+
|  W|  P|  L| WD| CWD| SPM| SWB|SWE|   SS|sales_forecast|
+---+---+---+---+----+----+----+---+-----+--------------+
| 31|  1| 01|600| 600|1000|1000|400|    0|           600|
| 32|  1| 01|700|1300|1000| 400|  0| -300|           400|
| 33|  1| 01|700|2000|1000|   0|  0|-1000|             0|
| 34|  1| 01|700|2700|1000|   0|  0|-1700|             0|
| 35|  1| 01|400|3100|1000|   